In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
test = pd.read_csv('./data/test/_annotations.csv')
pred = pd.read_csv('./data/test/test_pred.csv')



#test['filename'] = test['filename'].apply(lambda x : x.split('/')[-1])
#pred['filename'] = pred['filename'].apply(lambda x : x.split('/')[-1])


label_map = "labels.json"
output_path = "confusion_matrix.csv"

In [3]:
test.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,IMG_2526_jpeg_jpg.rf.003e1d1d41bcd204df731b85c...,768,1024,puffin,406,457,497,533
1,IMG_2526_jpeg_jpg.rf.003e1d1d41bcd204df731b85c...,768,1024,puffin,18,705,82,759
2,IMG_2526_jpeg_jpg.rf.003e1d1d41bcd204df731b85c...,768,1024,puffin,51,426,116,475
3,IMG_2526_jpeg_jpg.rf.003e1d1d41bcd204df731b85c...,768,1024,puffin,428,331,504,376
4,IMG_2526_jpeg_jpg.rf.003e1d1d41bcd204df731b85c...,768,1024,puffin,0,613,21,658


In [4]:
pred.describe()

,width,height,xmin,ymin,xmax,ymax,score
count,58400.000000,58400.000000,58400.000000,58400.000000,58400.000000,58400.000000,58400.000000
mean,807.232877,952.547945,356.585649,397.074491,454.742504,496.657478,0.231341
std,151.456206,114.832729,252.606125,256.383570,251.161402,271.233599,0.242151
min,576.000000,768.000000,0.000000,0.000000,18.399483,17.242199,0.021132
25%,768.000000,768.000000,143.912918,218.194679,246.173813,299.756104,0.081799
50%,768.000000,1024.000000,334.358311,363.184982,446.817719,460.412231,0.134894
75%,1024.000000,1024.000000,555.068298,548.464828,654.925171,671.185730,0.250338
max,1024.000000,1024.000000,1003.278809,1001.772034,1024.000000,1024.000000,0.992583


In [5]:
with open('labels.json') as json_file:
    categories = json.load(json_file)

In [6]:
IOU_THRESHOLD = 0.5
CONFIDENCE_THRESHOLD = 0.5


In [7]:
def compute_iou(groundtruth_box, detection_box):
    g_xmin, g_ymin, g_xmax, g_ymax = tuple(groundtruth_box)
    d_xmin, d_ymin, d_xmax, d_ymax = tuple(detection_box)
    
    xa = max(g_xmin, d_xmin)
    ya = max(g_ymin, d_ymin)
    xb = min(g_xmax, d_xmax)
    yb = min(g_ymax, d_ymax)
    #area of intersection
    intersection = max(0, xb - xa + 1) * max(0, yb - ya + 1)
    # "+1" because pixels coordinates are inclusive on both sites
    boxAArea = (g_xmax - g_xmin + 1) * (g_ymax - g_ymin + 1)
    boxBArea = (d_xmax - d_xmin + 1) * (d_ymax - d_ymin + 1)

    return intersection / float(boxAArea + boxBArea - intersection)

In [57]:
def process_detections(test,categories):
    #initializing confusion matrix
    confusion_matrix = np.zeros(shape=(len(categories) + 1, len(categories) + 1))
    #get the unique images
    file_unique = test['filename'].unique()
    #For each image
    for file in file_unique:
            print(file)
            #get all the gt data for that images
            test_df = test[test['filename']==file]
            test_df.reset_index(inplace = True, drop = True) 
            #get all the prediction made for that image
            pred_df = pred[pred['filename']==file]
            pred_df.reset_index(inplace = True, drop = True) 
            
            
            #filter the predictions with score >= CONFIDENCE_THRESHOLD    
            pred_class = pred_df[pred_df['score'] >= CONFIDENCE_THRESHOLD ]
           
            #gt and predicted bounding boxes in a list 
            groundtruth_boxes = test_df[['xmin','ymin','xmax','ymax']].values.tolist()
            detection_boxes = pred_class[['xmin','ymin','xmax','ymax']].values.tolist()
            print(groundtruth_boxes)
            print(detection_boxes)
            #initializing a list for storing matches
            matches = []   
            #for each gt box, calculate iou for each detection box and check if it is above IOU_THRESHOLD
            #If it is above IOU_THRESHOLD, then it is matching the gt box hence added the indexes of gt_box and d_box and iou
            #to matches. 
            for i in range(len(groundtruth_boxes)):
                for j in range(len(detection_boxes)):
                    iou = compute_iou(groundtruth_boxes[i], detection_boxes[j])
            
                    if iou > IOU_THRESHOLD:
                        matches.append([i, j, iou])
            
            #convert matches from list to array
            matches = np.array(matches)

            print(matches)
            
            
            if matches.shape[0] > 0:
                # Sort list of matches by descending IOU so we can remove duplicate detections
                # while keeping the highest IOU entry.
                matches = matches[matches[:, 2].argsort()[::-1][:len(matches)]]
                
                # Remove duplicate detections from the list.
                matches = matches[np.unique(matches[:,1], return_index=True)[1]]
                
                # Sort the list again by descending IOU. Removing duplicates doesn't preserve
                # our previous sort.
                matches = matches[matches[:, 2].argsort()[::-1][:len(matches)]]
                
                # Remove duplicate ground truths from the list.
                matches = matches[np.unique(matches[:,0], return_index=True)[1]]
            
            #for each gt box
            for i in range(len(groundtruth_boxes)):
                print(i)
                #if there are any matches and there any matches for that particular gt box
                if matches.shape[0] > 0 and matches[matches[:,0] == i].shape[0] == 1:
                    print("inside : ",i)
                    #TP
                    confusion_matrix[categories[test_df['class'][i]] - 1][categories[pred_class['class'].values[int(matches[matches[:,0] == i].tolist()[0][1])]] - 1] += 1
                else:
                    #FN
                    confusion_matrix[categories[test_df['class'][i]] - 1][confusion_matrix.shape[1] - 1] += 1
    
            for i in range(len(detection_boxes)):
               if matches.shape[0] > 0 and matches[matches[:,1] == i].shape[0] == 0:
                    #FP
                    confusion_matrix[confusion_matrix.shape[0] - 1][categories[pred_class['class'].values[i]] - 1] += 1

                
    return confusion_matrix

In [58]:
def display(confusion_matrix, test, categories, output_path):
    #list to save the results
    results = []
    #get all the classes
    class_uniq = test['class'].unique()
    #for each label
    for label in class_uniq:
        #getting id for that label
        class_id = int(float(categories[label]))-1
        name = label
        #TP + FN
        total_target = np.sum(confusion_matrix[class_id,:])
        
        #TP + FP
        total_predicted = np.sum(confusion_matrix[:,class_id])
        # TP/(TP+FP)
        precision = float(confusion_matrix[class_id, class_id] / total_predicted)
        #TP/(TP+FN)
        recall = float(confusion_matrix[class_id, class_id] / total_target)
    
        results.append({'category' : name, 'precision_@{}IOU'.format(IOU_THRESHOLD) : precision, 'recall_@{}IOU'.format(IOU_THRESHOLD) : recall})
    print(confusion_matrix)
    print(precision)
    print(recall)
    df = pd.DataFrame(results)
    print(df)
    df.to_csv(output_path)

In [61]:
confusion_matrix = process_detections(test,categories)
display(confusion_matrix, test, categories, output_path)

IMG_2526_jpeg_jpg.rf.003e1d1d41bcd204df731b85cea68781.jpg
[[406, 457, 497, 533], [18, 705, 82, 759], [51, 426, 116, 475], [428, 331, 504, 376], [0, 613, 21, 658], [20, 626, 72, 669], [319, 463, 386, 503], [536, 314, 591, 353], [0, 428, 21, 459], [98, 426, 131, 453]]
[[396.0765380859375, 452.8648681640625, 472.9885711669922, 524.31005859375], [516.3619995117188, 503.2830200195313, 615.2637634277344, 544.5776977539062], [182.70069122314453, 6.394916534423828, 258.6390151977539, 61.02005386352539], [424.9074096679688, 302.0805969238281, 505.4617309570313, 377.5178527832031], [19.45323657989502, 685.2073364257812, 102.91092681884766, 770.3745727539062], [85.06544494628906, 101.46498107910156, 147.41445922851562, 153.31919860839844], [396.0765380859375, 452.8648681640625, 472.9885711669922, 524.31005859375], [516.3619995117188, 503.2830200195313, 615.2637634277344, 544.5776977539062], [182.70069122314453, 6.394916534423828, 258.6390151977539, 61.02005386352539], [424.9074096679688, 302.0805

[[106, 654, 485, 810], [400, 578, 561, 683], [274, 257, 302, 359], [504, 550, 728, 632], [116, 296, 165, 371], [440, 290, 474, 360], [388, 299, 423, 365], [550, 265, 582, 331], [20, 254, 55, 319], [328, 480, 478, 536], [668, 490, 736, 535], [0, 282, 21, 327], [585, 278, 604, 322], [109, 276, 131, 313], [2, 253, 23, 290], [175, 295, 210, 332], [605, 290, 621, 325], [629, 453, 734, 486], [627, 306, 648, 336], [681, 304, 699, 331], [649, 304, 667, 331]]
[[104.9029769897461, 635.8864135742188, 442.3610229492188, 810.8078002929688], [408.661376953125, 589.199951171875, 557.8860168457031, 665.9896240234375], [104.55233001708984, 296.8387451171875, 160.17378616333008, 367.6910400390625], [264.99517822265625, 278.251708984375, 318.04042053222656, 362.25970458984375], [8.054150104522705, 256.0777893066406, 58.03146743774414, 327.6879577636719], [484.5478820800781, 200.0572052001953, 528.6997833251953, 268.6685791015625], [538.4871368408203, 271.78594970703125, 583.2556915283203, 341.58471679687

[[0.00000000e+00 3.50000000e+01 6.32255910e-01]
 [0.00000000e+00 7.80000000e+01 6.32255910e-01]
 [0.00000000e+00 1.21000000e+02 6.32255910e-01]
 ...
 [4.30000000e+01 1.82700000e+03 5.42090765e-01]
 [4.30000000e+01 1.87000000e+03 5.42090765e-01]
 [4.30000000e+01 1.91300000e+03 5.42090765e-01]]
0
inside :  0
1
2
inside :  2
3
inside :  3
4
5
inside :  5
6
inside :  6
7
inside :  7
8
9
inside :  9
10
inside :  10
11
inside :  11
12
inside :  12
13
inside :  13
14
inside :  14
15
inside :  15
16
17
inside :  17
18
inside :  18
19
inside :  19
20
inside :  20
21
inside :  21
22
23
inside :  23
24
inside :  24
25
inside :  25
26
inside :  26
27
inside :  27
28
inside :  28
29
inside :  29
30
inside :  30
31
inside :  31
32
inside :  32
33
inside :  33
34
inside :  34
35
36
37
38
39
40
41
42
43
inside :  43
44
IMG_2354_jpeg_jpg.rf.396e872c7fb0a95e911806986995ee7a.jpg
[[559, 589, 634, 660], [243, 452, 324, 522], [161, 365, 196, 411], [251, 369, 284, 405], [92, 436, 124, 470]]
[[252.62283325195

[[0.0000000e+00 2.0000000e+00 6.7102230e-01]
 [0.0000000e+00 2.0000000e+01 6.7102230e-01]
 [0.0000000e+00 3.8000000e+01 6.7102230e-01]
 ...
 [4.0000000e+01 7.5000000e+02 8.2635857e-01]
 [4.0000000e+01 7.6800000e+02 8.2635857e-01]
 [4.0000000e+01 7.8600000e+02 8.2635857e-01]]
0
inside :  0
1
2
3
inside :  3
4
5
inside :  5
6
inside :  6
7
8
9
10
11
12
inside :  12
13
14
inside :  14
15
16
17
18
19
inside :  19
20
21
inside :  21
22
23
24
25
inside :  25
26
27
28
inside :  28
29
30
31
32
33
34
inside :  34
35
36
inside :  36
37
38
39
40
inside :  40
41
42
43
IMG_2319_jpeg_jpg.rf.6e20bf97d17b74a8948aa48776c40454.jpg
[[379, 294, 457, 412], [349, 476, 458, 567], [660, 243, 707, 310], [702, 241, 730, 301], [530, 244, 576, 302], [730, 252, 752, 295], [626, 255, 655, 280], [470, 163, 508, 197]]
[[377.4983367919922, 299.59478759765625, 450.2940673828125, 402.8650512695313], [375.5689086914063, 476.12152099609375, 459.1961059570313, 572.9711303710938], [523.6658935546875, 242.7003326416016, 577.

0
inside :  0
1
inside :  1
2
inside :  2
IMG_2468_jpeg_jpg.rf.c933cc14c99b11a90413a1490d4556db.jpg
[[226, 119, 420, 336], [0, 470, 109, 649], [184, 631, 403, 802], [394, 614, 619, 771], [409, 83, 552, 223], [695, 402, 767, 533], [58, 953, 230, 1023], [711, 813, 767, 928]]
[[396.8048400878906, 79.74668884277344, 558.4000396728516, 214.30992126464844], [227.86871337890625, 143.94842529296875, 419.584716796875, 333.7803649902344], [398.411865234375, 629.4783325195312, 613.9959869384766, 752.3960571289062], [0.0, 465.5594787597656, 109.3788070678711, 655.0358276367188], [177.6514663696289, 614.1377563476562, 402.7646484375, 787.3926391601562], [692.5304260253906, 405.6120910644531, 767.8184967041016, 512.34375], [737.052978515625, 777.1123046875, 768.0, 1011.8779296875], [725.7215881347656, 827.9154052734375, 768.0, 919.315185546875], [320.3394470214844, 949.0422973632812, 430.6975708007813, 1021.6353149414062], [396.8048400878906, 79.74668884277344, 558.4000396728516, 214.30992126464844]

In [65]:
confusion_matrix

array([[1.090e+02, 3.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00,
        0.000e+00, 1.360e+02],
       [0.000e+00, 1.230e+02, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 3.100e+01],
       [0.000e+00, 0.000e+00, 2.600e+01, 0.000e+00, 2.000e+00, 0.000e+00,
        0.000e+00, 5.400e+01],
       [5.000e+00, 0.000e+00, 0.000e+00, 2.300e+01, 0.000e+00, 0.000e+00,
        0.000e+00, 1.000e+01],
       [0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 9.000e+00, 0.000e+00,
        0.000e+00, 2.600e+01],
       [0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 1.200e+01,
        0.000e+00, 2.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        9.000e+00, 2.000e+00],
       [2.247e+03, 3.776e+03, 4.230e+02, 3.220e+02, 1.070e+02, 1.910e+02,
        2.800e+01, 0.000e+00]])